In [1]:
import random
import numpy as np
import pandas as pd

df = pd.DataFrame([[0, 0, 0, 0], [1, 0, 0, 0], [1, 1, 0, 1], [1, 0, 1, 1], [1, 1, 1, 1]],
                  columns = ['x0', 'x1', 'x2', 'y'])

In [2]:
def get_errors(feature_matrix, label, coefficients, threshold, upper, lower): 
    
    count = 0
    pred_dict = {}
    error_ids = []
    
    for n in range(len(coefficients)): # calculando e limiarizando predições 
        predicted = round(sum(feature_matrix.iloc[n] * coefficients))
        
        if predicted >= threshold:
            predicted = upper
        else:
            predicted = lower
            
        pred_dict[count] = {'predicted' : predicted, 'actual' : label.iloc[n]}
        count += 1
        
    for key in pred_dict.keys():                        # comparando predições com valores verdadeiros
        if pred_dict[key]['predicted'] != pred_dict[key]['actual']:
            error_ids.append(key)  
            
    if len(error_ids) == 0:                       # condição de parada (convergência)
        return None, coefficients, pred_dict      
    else:
        return min(error_ids), coefficients, pred_dict 
    # caso haja erro, retorna por onde deve começar a 'revisitar'

def coefficient_optimization(error_id, coef_list, pred_dict, learning_rate, feature_matrix, label): 
    # 'revisita' (na primeira predição errada)
    if error_id == None: # interpreta condição de parada implementada na função anterior
        return 'stop', coef_list

    bad_row = feature_matrix.iloc[error_id] # otimiza coeficiente, à partir da observação errada
    new_coefs = []
    for n in range(len(bad_row)): # 'bad_row' é uma 'series'
        new_coefs.append(coef_list[n] + learning_rate * bad_row[n] * \
                         (label.iloc[error_id] - pred_dict[error_id]['predicted']))    
        
    return 'go', new_coefs

def perceptron(feature_matrix, label, coefficients, learning_rate, threshold, upper, lower, max_epoch):
    
    go = 'go'
    epoch = 0
    while True:

        if go == 'go':
            error_id, coef_list, pred_dict = get_errors(feature_matrix, label, coefficients, 
                                                        threshold, upper, lower)
            go, new_coefs = coefficient_optimization(error_id, coef_list, pred_dict,
                                                     learning_rate, feature_matrix, label) 
            coefficients = new_coefs
            epoch += 1

            if go == 'stop' or epoch == max_epoch:

                if epoch == max_epoch and go != 'stop':
                    print('max_epoch reached, no convergence', new_coefs)
                    break
                    
                elif epoch != max_epoch and go == 'stop':
                    print('convergence achieved', new_coefs)    
                    break
                    
                elif go == 'stop' and epoch == max_epoch:
                    print('convergence achieved, max_epoch reached', new_coefs)
                    break

In [3]:
# k -> número de variáveis explicativas
# posso usar o random com 'range' ?

# ínidice do dataframe 'ordenado' (não sei se convém fazer o tratamento, esperar que a função funcione caso o
# índice do dataframe estivesse em outro formato)

# acho que está pronto, posso arrumar nomes de variáveis e retorno da função, apresentar predições ?

seed = 123
random.seed(a = seed)
coefficients = random.choices(population = np.arange(start = -1, stop = 1, step = 0.001), k = 3)

perceptron(coefficients = coefficients,
           threshold = 0.5,
           upper = 1,
           lower = 0,
           learning_rate = 0.1,
           feature_matrix = df[['x0', 'x1', 'x2']],
           label = df['y'],
           max_epoch = 7)

max_epoch reached, no convergence [-0.19600000000000004, -0.12599999999999995, -0.18599999999999928]
